In [24]:
import json
import gzip

In [25]:
with gzip.open('user_profile/20180411/part-00102.gz', 'rb') as f:
    lines = f.readlines()

In [26]:
len(lines)

35

In [27]:
def parse_old_profile(line):
    item = eval(line.decode('utf-8'))
    if len(item) == 3:
        uid = item[0]
        topic_dic = item[1]
        day_tag = int(item[2])
        if uid != "" and  uid != "-" and len(uid) > 6:
            return (uid,('old',topic_dic,day_tag))


In [28]:
old_profile = [parse_old_profile(line) for line in lines]

In [29]:
len(old_profile)

35

In [43]:
old_profile[0][1][1].keys()

dict_keys(['1C1', '21', '141', '51', '121', 'C1'])

In [41]:
old_profile[0][1][1]['121']

[(-650837167, 4.859),
 (1513139701, 1.0),
 (-1920959643, 1.0),
 (-1501408193, 1.0)]

In [44]:
old_profile

[('CQk4MDU4YTM4OWEyYWE4YTg1CU1LSk5XMTgzMjEwMDAxOTA%3D',
  ('old',
   {'121': [(-650837167, 4.859),
     (1513139701, 1.0),
     (-1920959643, 1.0),
     (-1501408193, 1.0)],
    '141': [(-44974079, 1.0),
     (1529537571, 1.0),
     (-962259995, 1.0),
     (1845123270, 1.0),
     (-1328941194, 1.0),
     (1563709544, 1.0),
     (-742406164, 1.0),
     (-1907341683, 1.0),
     (-2113386161, 1.0),
     (1321926101, 1.0),
     (1940695574, 1.0),
     (-856998409, 1.0),
     (-289071813, 1.0),
     (992178074, 1.0),
     (-495567857, 1.0),
     (1452042065, 0.972),
     (-228285214, 0.944),
     (549621338, 0.944)],
    '1C1': [(-1727498363, 1.0),
     (1405008138, 1.0),
     (-929164781, 1.0),
     (721815963, 1.0),
     (-915773796, 1.0),
     (427453106, 1.0),
     (391444937, 1.0),
     (-2068415939, 1.0),
     (1513139701, 1.0),
     (-1332907831, 1.0),
     (1729907539, 1.0),
     (-1290266279, 1.0),
     (1081907422, 1.0),
     (-1057504655, 1.0),
     (391627378, 1.0),
     (-88552

In [45]:
def cal_topic_weight(topic_data):
    last_uid = ''
    cur_uid = ''
    new_topic = {}
    old_topic = {}
    day_tag = 0
    factor_damping = 0.9716
    for line in topic_data:
        cur_uid = line[0]
        
        if last_uid != cur_uid and last_uid != '': 
            if isinstance(new_topic, str):
                new_topic = eval(new_topic)
            if isinstance(old_topic, str):
                old_topic = eval(old_topic)

            factor = factor_damping ** (day_tag)
            topic_dic = {}
            if len(old_topic) > 0:
                for label_type in old_topic:
                    if label_type not in topic_dic:
                        topic_dic[label_type] = {}
                    for item in old_topic[label_type]:
                        topicid = item[0]
                        score = float(item[1])
                        if len(new_topic) <= 0 or label_type not in new_topic:
                            topic_dic[label_type][topicid] = round(score,3)
                        else:
                            topic_dic[label_type][topicid] = round(score * factor,3) 
            
            if len(new_topic) > 0 :
                day_tag = 0
                for label_type in new_topic:
                    if label_type not in topic_dic:
                        topic_dic[label_type] = {}
                    if isinstance(new_topic[label_type],str):
                        new_topic[label_type] = eval(new_topic[label_type])
                    for item in new_topic[label_type]:
                        topicid = item
                        score = 1
                        if topicid not in topic_dic[label_type]:
                            topic_dic[label_type][topicid] =  0
                        topic_dic[label_type][topicid] += round(score,3)      
            
            if len(topic_dic) > 0 :
                for label_type in topic_dic:
                    try:
                        topic_list = sorted(topic_dic[label_type].items(), key=lambda x: x[1], reverse=True)
                    except:
                        yield('ERROR1',topic_dic[label_type])
                    for idx in range(len(topic_list)):
                        if topic_list[idx][1] < 0.05:
                            break
                    if len(topic_list) > 0:
                        topic_dic[label_type] = topic_list[0:idx+1]
                uid = last_uid
                if len(topic_dic) > 0: 
                    yield(uid,topic_dic,day_tag)
            new_topic = {}
            old_topic = {}
            day_tag = 0

        if line[1][0] == 'new':
            new_topic = line[1][1]
        if line[1][0] == 'old':
            old_topic = line[1][1]
            day_tag = int(line[1][2]) + 1
        last_uid = cur_uid

    
    factor = factor_damping ** (day_tag)
    topic_dic = {}

    if isinstance(new_topic, str):
        new_topic = eval(new_topic)
    if isinstance(old_topic, str):
        old_topic = eval(old_topic)
    if len(old_topic) > 0 and isinstance(old_topic, dict):
        for label_type in old_topic:
            if label_type not in topic_dic:
                topic_dic[label_type] = {}
            for item in old_topic[label_type]:
                topicid = item[0]
                score = float(item[1])
                if len(new_topic) <= 0 :
                    topic_dic[label_type][topicid] = round(score,3)
                else:
                    topic_dic[label_type][topicid] = round(score * factor,3) 
    if len(new_topic) > 0 and isinstance(new_topic, dict):
        for label_type in new_topic:
            if label_type not in topic_dic:
                topic_dic[label_type] = {}
            if isinstance(new_topic[label_type], str):
                new_topic[label_type] = eval(new_topic[label_type])
            for item in new_topic[label_type]:
                topicid = item
                score = 1
                if topicid not in topic_dic[label_type]:
                    topic_dic[label_type][topicid] = 0
                topic_dic[label_type][topicid] += round(score,3)

    if len(topic_dic) > 0:
        for label_type in topic_dic:
            topic_list = sorted(topic_dic[label_type].items(), key=lambda x: x[1], reverse=True)
            for idx in range(len(topic_list)):
                if topic_list[idx][1] < 0.05:
                    break
            if len(topic_list) > 0:
                topic_dic[label_type] = topic_list[0:idx+1]

    if len(topic_dic) > 0: 
        uid = last_uid
        return (uid,topic_dic,day_tag)    
